In [7]:
import pandas as pd 

# Creating DFs that have some mistakes and differences
patients = pd.DataFrame({
    "patient_id" : [101, 102, 103, 104, 105, 105],
    "name" : ["Ana", "Ben", "Cara", "Dan", "JJ", "JJ"],
    "age" : [34, 58, 45, 50, 29, 29],
    "insurance" : ["Private", "Medicare", "Private", "None", "Private", "Private"]
})

visits = pd.DataFrame({
    "visit_id": [1, 2, 3, 4, 5, 6],
    "patient_id": [101, 102, 106, 105, 105, 107],
    "visit_cost": [120, 250, 180, 300, 90, 200],
    "clinic": ["A", "A", "B", "A", "B", "A"]
})

patients, visits

(   patient_id  name  age insurance
 0         101   Ana   34   Private
 1         102   Ben   58  Medicare
 2         103  Cara   45   Private
 3         104   Dan   50      None
 4         105    JJ   29   Private
 5         105    JJ   29   Private,
    visit_id  patient_id  visit_cost clinic
 0         1         101         120      A
 1         2         102         250      A
 2         3         106         180      B
 3         4         105         300      A
 4         5         105          90      B
 5         6         107         200      A)

In [10]:
# Checking the keys for patients DF
patients["patient_id"].value_counts()


patient_id
105    2
101    1
102    1
103    1
104    1
Name: count, dtype: int64

In [ ]:
# Checking the keys for visits DF
visits["patient_id"].value_counts()

patient_id
105    2
101    1
102    1
106    1
107    1
Name: count, dtype: int64

In [ ]:
# Using Inner Join to join 
# the matching records

inner_join = pd.merge(visits, 
                      patients,
                      on = "patient_id",
                      how = "inner"
                     )

inner_join


,visit_id,patient_id,visit_cost,clinic,name,age,insurance
0,1,101,120,A,Ana,34,Private
1,2,102,250,A,Ben,58,Medicare
2,4,105,300,A,JJ,29,Private
3,4,105,300,A,JJ,29,Private
4,5,105,90,B,JJ,29,Private
5,5,105,90,B,JJ,29,Private


In [14]:
# Using a left join keeps 
# all the rows from the left table

left_join = pd.merge(
    visits,
    patients,
    on = "patient_id",
    how = "left"
)

left_join

,visit_id,patient_id,visit_cost,clinic,name,age,insurance
0,1,101,120,A,Ana,34.0,Private
1,2,102,250,A,Ben,58.0,Medicare
2,3,106,180,B,NaN,NaN,NaN
3,4,105,300,A,JJ,29.0,Private
4,4,105,300,A,JJ,29.0,Private
5,5,105,90,B,JJ,29.0,Private
6,5,105,90,B,JJ,29.0,Private
7,6,107,200,A,NaN,NaN,NaN


In [15]:
# Data Quality Check 
# used to find the missing patients

missing_patients = left_join[left_join["name"].isna()]
missing_patients

,visit_id,patient_id,visit_cost,clinic,name,age,insurance
2,3,106,180,B,NaN,NaN,NaN
7,6,107,200,A,NaN,NaN,NaN


In [17]:
#Finding the duplicate keys

left_join["patient_id"].value_counts()

patient_id
105    4
101    1
102    1
106    1
107    1
Name: count, dtype: int64

In [ ]:
# Patients that generate the most visits
visit_counts = (
    left_join
    .groupby("patient_id")
    .size()
    .reset_index(name="num_visits")
    .sort_values("num_visits", ascending=False)
)

visit_counts


,patient_id,num_visits
2,105,4
0,101,1
1,102,1
3,106,1
4,107,1


In [ ]:
# Validating the sizes of the dataframes
# and the join results

print("visits rows: ", len(visits))
print("left join rows: ", len(left_join))
print("inner join rows: ", len(inner_join))

visits rows:  6
left join rows:  8
inner join rows:  6


In [ ]:
# Fixing the duplicate, 
# and creating a clean 
# left join using the validate


patients_dedup = patients.drop_duplicates(subset = "patient_id", 
                                          keep = "first")

left_join_clean = pd.merge(
    visits,
    patients_dedup,
    on = "patient_id",
    how = "left",
    validate = "m:1"
)

left_join_clean

,visit_id,patient_id,visit_cost,clinic,name,age,insurance
0,1,101,120,A,Ana,34.0,Private
1,2,102,250,A,Ben,58.0,Medicare
2,3,106,180,B,NaN,NaN,NaN
3,4,105,300,A,JJ,29.0,Private
4,5,105,90,B,JJ,29.0,Private
5,6,107,200,A,NaN,NaN,NaN
